# Make NSRDB Projection Data
Collect NSRDB data and make projection files per region

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from glob import glob
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import logging
from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt
import xarray as xr

from rex import Resource, MultiYearNSRDB, init_logger
from region_classifier import RegionClassifier

from sup3r.preprocessing.data_handling.base import DataHandler
from sup3r.preprocessing.data_handling import DataHandlerNCforCC
from sup3r.preprocessing.data_handling import DataHandlerNCforCCwithPowerLaw
from sup3r.bias.bias_calc import SkillAssessment

from make_projection_summaries_cmip import get_fps, get_targets_shapes, make_summary_files, FEATURES, MODELS, TAGS, REGIONS

DataHandlerNCforCC.CHUNKS = {'time': None, 'lat': None, 'lon': None}

In [ ]:
logger = logging.getLogger(__name__)
init_logger(__name__)
init_logger('make_projection_summaries')
init_logger('sup3r', log_level='DEBUG')

In [ ]:
res = MultiYearNSRDB('/kfs2/pdatasets/NSRDB/current/nsrdb_2*.h5')

In [ ]:
nsrdb_dset = 'ghi'
out_dset = 'rsds'

In [ ]:
def read(i_split_gids):
    with MultiYearNSRDB('/kfs2/pdatasets/NSRDB/current/nsrdb_2*.h5') as res:
        out = res[nsrdb_dset, :, i_split_gids].sum(axis=1).astype(np.float32)
    return out

In [ ]:
for region, rstates in REGIONS.items():
    region = region.lower().replace(' ', '_')

    if region != 'conus':
        mask = res.meta['state'].isin(rstates)
        assert len(res.meta.loc[mask, 'state'].unique()) == len(rstates)
        rgids = np.where(mask)[0]
    
        fp_out = f'./projections/{region}_nsrdb_{out_dset}.csv'
        if not os.path.exists(fp_out):
            logger.info(f'Working on {region}')
            split_gids = np.array_split(rgids, 100)
            futures = []
            out = 0
            with ProcessPoolExecutor() as exe:
                for i_split_gids in split_gids:
                    future = exe.submit(read, i_split_gids)
                    futures.append(future)
                
                for i, (future, i_split_gids) in enumerate(zip(futures, split_gids)):
                    out += future.result()
                    logger.info('Finished future {} out of {}'.format(i+1, len(futures)))
            
            out /= len(rgids)
            
            df = pd.DataFrame({'NSRDB': out}, index=res.time_index)
            df.to_csv(fp_out)
            logger.info(f'Saved: {fp_out}')


In [ ]:
df_conus = None
idfs = {}
n_total = 0
for region, rstates in REGIONS.items():
    region = region.lower().replace(' ', '_')

    if region != 'conus':
        mask = res.meta['state'].isin(rstates)
        assert len(res.meta.loc[mask, 'state'].unique()) == len(rstates)
        rgids = np.where(mask)[0]
    
        fp_out = f'./projections/{region}_nsrdb_{out_dset}.csv'

        idf = pd.read_csv(fp_out, index_col=0)
        idf.index = pd.to_datetime(idf.index)
        idfs[region] = idf.copy()
        idf *= len(rgids)
        n_total += len(rgids)

        if df_conus is None:
            df_conus = idf.copy()
        else:
            df_conus += idf.copy()

df_conus /= n_total

In [ ]:
df_conus.to_csv(f'./projections/conus_nsrdb_{out_dset}.csv')
df_conus

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
df_conus.rolling(365*10).mean().iloc[::1000].plot(ax=ax)
idfs['south'].rolling(365*10).mean().iloc[::1000].plot(ax=ax)